In [243]:
import random
import numpy as np
from scipy.sparse import dia_array
from scipy.sparse.linalg import spsolve


# import pandas as pd
import torch
import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader

# from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

## Samples Generator

In [244]:
def tensor2matrix(t):
    ni, nj = t[0].shape

    diag_ni_u = torch.flatten(t[0].transpose(0,1))
    diag_1_u = torch.flatten(t[1].transpose(0,1))
    diag_1_l = torch.flatten(t[2].transpose(0,1))
    diag_ni_l = torch.flatten(t[3].transpose(0,1))

    n_diag = ni*nj
    main_diag = np.ones(n_diag)
    data = np.array([diag_ni_l,diag_1_l, main_diag, diag_1_u,diag_ni_u])
    offsets = np.array([-ni,-1, 0, 1,ni])
    m = dia_array((data, offsets), shape=(n_diag, n_diag))

    return m

def make_tensor(number_generator,
                args={},
                symmetric=True,
                off_diagonal_abs_mean=0.5,
                grid_size=(5, 5)):

    #Generate values
    ni, nj = grid_size
    n_diag_1 = (ni-1)*nj
    n_diag_ni = ni*(nj-1)

    diag_1_u = [number_generator(**args) for _ in range(n_diag_1)]
    diag_ni_u = [number_generator(**args) for _ in range(n_diag_ni)]

    if symmetric:
        diag_1_l = diag_1_u
        diag_ni_l = diag_ni_u
    else:
        diag_1_l = [number_generator(**args) for _ in range(n_diag_1)]
        diag_ni_l = [number_generator(**args) for _ in range(n_diag_ni)]

    #To tensor
    diag_1_u = torch.tensor(diag_1_u).float()
    diag_1_l = torch.tensor(diag_1_l).float()
    diag_ni_u = torch.tensor(diag_ni_u).float()
    diag_ni_l = torch.tensor(diag_ni_l).float()

    #Scale off main diagonal
    off_diagonal = torch.cat([diag_ni_u, diag_1_u, diag_1_l, diag_ni_l])
    off_diagonal = torch.abs(off_diagonal)
    mean_abs = torch.mean(off_diagonal)
    alpha = off_diagonal_abs_mean / mean_abs
    diag_1_u = torch.mul(diag_1_u, alpha)
    diag_1_l = torch.mul(diag_1_l, alpha)
    diag_ni_u = torch.mul(diag_ni_u, alpha)
    diag_ni_l = torch.mul(diag_ni_l, alpha)

    #Adjust zeroes
    diag_1_u = torch.reshape(diag_1_u, (ni-1,nj))
    diag_1_u = torch.cat([torch.zeros(1,nj), diag_1_u], dim=0)

    diag_1_l = torch.reshape(diag_1_l, (ni-1,nj))
    diag_1_l = torch.cat([diag_1_l, torch.zeros(1,nj)], dim=0)

    diag_ni_l = torch.reshape(diag_ni_l, (ni,nj-1))
    diag_ni_l = torch.cat([diag_ni_l, torch.zeros(ni,1)], dim=1)

    diag_ni_u = torch.reshape(diag_ni_u, (ni,nj-1))
    diag_ni_u = torch.cat([torch.zeros(ni,1), diag_ni_u], dim=1)

    t = torch.cat([diag_ni_u.unsqueeze(0),
                    diag_1_u.unsqueeze(0),
                    diag_1_l.unsqueeze(0),
                    diag_ni_l.unsqueeze(0)],
                    dim=0)
    return t

In [245]:
def get_sample(number_generator,
               args={},
               symmetric=True,
               off_diagonal_abs_mean=0.5,
               grid_size=(5, 5)):

    A = make_tensor(number_generator=number_generator,
                    args=args,
                    symmetric=symmetric,
                    off_diagonal_abs_mean=off_diagonal_abs_mean,
                    grid_size=grid_size)

    ni, nj = grid_size
    n_diag = ni*nj
    x_true = [number_generator(**args) for _ in range(n_diag)]

    A_mat = tensor2matrix(A)
    b = A_mat.dot(x_true)
    b = torch.tensor(b, dtype=torch.float32)
    b = torch.reshape(b, (nj,ni)).transpose(0,1)

    X = torch.cat([A, b.unsqueeze(0)], dim=0)
    x_true = torch.tensor(x_true).float()
    y = torch.reshape(x_true, (nj,ni)).transpose(0,1)
    return (X, y)

In [398]:
def check_sample(X, y, result='max', print_cond_number=False, verbose=False):

    def _error_from_y(X,y):
        custom_weights = np.array([
            [[0, 0, 0],
             [0, 1, 0],
             [0, 0, 0]],
            [[0, 0, 0],
             [0, 0, 1],
             [0, 0, 0]],
            [[0, 0, 0],
             [0, 0, 0],
             [0, 1, 0]],
            [[0, 1, 0],
             [0, 0, 0],
             [0, 0, 0]],
            [[0, 0, 0],
             [1, 0, 0],
             [0, 0, 0]],
            ], dtype=np.float32)
        custom_weights_tensor = torch.tensor(custom_weights).unsqueeze(0)  # Add batch dimensions
        conv_layer = nn.Conv2d(in_channels=5, out_channels=1, kernel_size=3, stride=1, padding=1, bias=False)
        conv_layer.weight = nn.Parameter(custom_weights_tensor, requires_grad=False)

        ni, nj = X[0].shape
        X_ = torch.cat([torch.ones((ni,nj)).unsqueeze(0), X[:4]])
        residuals = conv_layer(X_ * y) - X[-1]
        residuals = torch.flatten(residuals.transpose(0,1))
        return residuals

    def _error_from_solve(X):
        A = tensor2matrix(X[:4])
        b = torch.flatten(X[-1].transpose(0,1)).tolist()
        y_aprox = spsolve(A.tocsr(), b)
        residuals = A.dot(y_aprox) - b
        y_aprox = torch.tensor(y_aprox)
        residuals = torch.tensor(residuals)
        return residuals, y_aprox

    def _get_result(output_tensor, reference_tensor):
        if result == 'sum_abs':
            return float(torch.sum(torch.abs(output_tensor)))
        if result == 'max':
            return float(torch.max(torch.abs(output_tensor)))
        if result == 'sum2':
            return float(torch.sum(torch.square(output_tensor)))
        if result == 'max_rel':
            return float(torch.max(torch.abs(output_tensor) / torch.abs(reference_tensor)))
        raise ValueError(f'Invalid option: {result}.')


    y_ = torch.flatten(y.transpose(0,1))
    if print_cond_number and verbose:
        A = tensor2matrix(X[:4])
        cond_num = np.linalg.cond(A)
        print(f'Matrix condition number: {cond_num:0.4g}')

    residuals_from_y = _error_from_y(X,y)
    error_from_y = _get_result(residuals_from_y, y_)

    residuals_from_solve, y_aprox = _error_from_solve(X)
    error_from_solve = _get_result(residuals_from_solve, y_)
    error_delta_y = _get_result(y_ - y_aprox, y_)

    if verbose:
        print(f'Result using provided y: {error_from_y:0.4g}')
        print(f'Result using sparse solver: {error_from_solve:0.4g}')
        print(f'Difference in y vectors: {error_delta_y:0.4g}')

    # return error_from_y, error_from_solve, error_delta_y
    return residuals_from_y, residuals_from_solve, y_aprox, y_

In [399]:
ni, nj = 300, 300
number_generator = random.uniform
args = {'a':1,'b':5}

X, y = get_sample(
    number_generator=number_generator,
    args=args,
    grid_size=(ni,nj),
    off_diagonal_abs_mean=0.5,
    symmetric=False)
print(f'X: {X.shape}')
print(f'Dense matrix: {tensor2matrix(X[:4]).shape}')
print(f'y: {y.shape}')

check_sample(X,y, 'max_rel', verbose=True) #max, sum_abs, sum2, max_rel

X: torch.Size([5, 300, 300])
Dense matrix: (90000, 90000)
y: torch.Size([300, 300])
Result using provided y: 1.745e-06
Result using sparse solver: 1.262e-13
Difference in y vectors: 0.0002761


(tensor([-2.3842e-07, -4.7684e-07,  0.0000e+00,  ...,  0.0000e+00,
          9.5367e-07, -4.7684e-07]),
 tensor([ 0.0000e+00,  0.0000e+00, -8.8818e-16,  ...,  0.0000e+00,
         -8.8818e-16,  0.0000e+00], dtype=torch.float64),
 tensor([1.1723, 2.8461, 1.3480,  ..., 1.6731, 3.2442, 3.7130],
        dtype=torch.float64),
 tensor([1.1723, 2.8461, 1.3479,  ..., 1.6731, 3.2442, 3.7130]))

## Neural Network Definition